In [1]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
from tensorflow import keras
from sklearn.model_selection import train_test_split # 훈/테 분할기 여전히 사용

(train_input, train_target), (test_input, test_target) = \
    keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0 # 표준화도 여전히 이렇게

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [ ]:
# 매번 모델 만들기 귀찮으니까, 함수로 만들자.
def model_fn(a_layer=None):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    model.add(keras.layers.Dense(100, activation='relu'))
    if a_layer: # 은닉층 뒤에 원하는 층을 더 추가할 수 있게 설정
        model.add(a_layer)
    model.add(keras.layers.Dense(10, activation='softmax')) # 마지막에 동일한 출력층
    return model

In [ ]:
model = model_fn()

model.summary()

In [ ]:
# 자 이번엔, 똑같이 fit 하는데 그 결과를 history에 담아보겠음
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=5, verbose=0)

In [ ]:
# 결과 담긴 history객체에서 key 값 꺼내보니 [손실, 정확도]
print(history.history.keys())

In [ ]:
# 그래프 그려보자
import matplotlib.pyplot as plt

plt.plot(history.history['loss']) # 인덱스 0부터, 그에 해당하는 loss 값 그려줘
plt.xlabel('epoch') # 에포크가 0부터 시작하니까 인덱스랑 마찬가지
plt.ylabel('loss')
plt.show()

In [ ]:
plt.plot(history.history['accuracy']) # 이번엔 accuracy 그려줘
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
model = model_fn() # 기본 모델에다가
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=20, verbose=0)

In [ ]:
plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
model = model_fn()
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target)) #fit의 매개변수로, 검증세트의 결과도 반환하도록 설정 가능함.

In [ ]:
print(history.history.keys()) 
#dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model = model_fn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

드롭아웃
- 신경망 모델에만 있는 규제 방법으로, 딥러닝의 아버지 Geoffrey Hinton이 소개함.
- 층에 있는 유닛을 다 훈련하지 않고 일부를 랜덤하게 off해서 훈련 성능을 낮춤

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))#비율 설정

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=10, verbose=0,
                    validation_data=(val_scaled, val_target))

In [ ]:
model.save_weights('model-weights.h5') # 모델을 저장해두자. 가중치;만

In [ ]:
model.save('model-whole.h5') # 가중치랑 모델 구조까지 싹 다 저장하려면 이렇게!

In [ ]:
!ls -al *.h5 # 잘 만들어졌나 셀 명령으로 확인

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))# 새 모델 만들고

model.load_weights('model-weights.h5') # 만들었던 가중치 반영

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))

model.load_weights('model-weights.h5')
import numpy as np

val_labels = np.argmax(model.predict(val_scaled), axis=-1) # .argmax() : predict가 출력한 확률들 중에 가장 큰 값을 뽑기 위해 넘파이 활용함.
print(np.mean(val_labels == val_target)) # 모델이 구한 예측클래스와 실제 타깃의 정답을 비교해서, 맞춘 비율을 본다..

In [ ]:
model = keras.models.load_model('model-whole.h5')

model.evaluate(val_scaled, val_target)

콜백
- 훈련 과정 중간에 특정 작업을 수행하게 해주는 객체: keras.callbacks 패키지 아래에 다양한 클래스들이 있음
- -> 콜백 객체를 만들어두고, fit()할 때 callback 매개변수로 전달하면 됨.

ModelCheckpoint(): 가장 자주 사용되는 콜백으로, 에포크마다 모델을 저장해줌
save_best_only=True : 손실이 가장 낮은 모델만 저장하도록 하는 설정.

In [ ]:
model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5',
                                                save_best_only=True)# 콜백으로 체크포인트 객체 만들어주고

model.fit(train_scaled, train_target, epochs=20, verbose=0,
          validation_data=(val_scaled, val_target),
          callbacks=[checkpoint_cb]) # fit할 때 콜백 매개변수에 넣어주면 됨.

In [ ]:
model = keras.models.load_model('best-model.h5')

model.evaluate(val_scaled, val_target)

EarlyStopping 콜백
- 자, 최적의 에포크 찾는 건 오케이. 그런데, 그 최적의 모델 찾을 때 에포크를 무작정 높게 설정해놓고 찾으면, 불필요하게 오랫동안 훈련을 계속함.
- EarlyStopping() : 과대적합이 시작되면 훈련을 알아서 조기종료 해주는 콜백.
    patience : 검증세트 성능이 좋아지지 않더라도 참고 기다릴 에포크 횟수 설정.
    restore_best_weights=True : 훈련동안 가장 손실 낮았던 최적 가중치로 돌리는 설정.

In [ ]:
model = model_fn(keras.layers.Dropout(0.3)) # 똑같이 모델 만드는데
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5',
                                                save_best_only=True) # 콜백으로 체크포인트 객체 만들어주고
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,
                                                  restore_best_weights=True) #조기 종료 콜백을 추가

history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb]) # 여기서도 추가

이러면 이제 fit()에서 epochs를 마음놓고 크게 설정해도 괜찮음^^
.stopped_epoch : 몇 번째 에포크에서 조기종료 했는지 저장되어 있는 속성.
→ patience 설정했던 것과 같이 생각해보면 최상의 에포크가 언제인지 나옴!
ex) 2번 참았는데, 11번째 에포크에서 끝났단 건 9번째 에포크가 최상의 모델이란 거네~

In [ ]:
print(early_stopping_cb.stopped_epoch)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model.evaluate(val_scaled, val_target)